In [1]:
# Install required packages
!pip install -q transformers

In [2]:
# Import required packages
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from tqdm.notebook import tqdm

from transformers import BertConfig, BertTokenizer
from transformers import TFBertModel, TFBertForSequenceClassification
from transformers import glue_convert_examples_to_features

import tensorflow as tf

from transformers import TFBertModel, TFBertForSequenceClassification, BertModel
from transformers.modeling_tf_utils import get_initializer

from sklearn.model_selection import train_test_split
import sklearn.metrics as skm

2025-04-18 18:28:42.625754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745000922.828938      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745000922.883391      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
!git clone https://github.com/vkiani/ShortPersianEmo/

Cloning into 'ShortPersianEmo'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 67 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (67/67), 1.97 MiB | 24.59 MiB/s, done.
Resolving deltas: 100% (18/18), done.


### Load the data using Pandas

In [4]:
emotion_dataset_train = pd.read_excel('ShortPersianEmo/Data/ShortPersianEmo/train_fa.xlsx', header=None).dropna()
x_train = emotion_dataset_train[0]
y_train = emotion_dataset_train[1]

emotion_dataset_test = pd.read_excel('ShortPersianEmo/Data/ShortPersianEmo/test_fa.xlsx', header=None).dropna()
x_test = emotion_dataset_test[0]
y_test = emotion_dataset_test[1]

In [5]:
from collections import Counter
cnt = Counter(y_train)
cnt = dict(cnt)
print(cnt)

{'SAD': 845, 'OTHER': 1263, 'HAPPY': 1462, 'ANGRY': 1012, 'FEAR': 342}


### Normalization / Preprocessing

In [6]:
data_train = emotion_dataset_train[[0, 1]]
data_train.columns = ['comment', 'label']

data_test = emotion_dataset_test[[0, 1]]
data_test.columns = ['comment', 'label']

## Train,Validation,Test split

In [7]:
test = data_test
train = data_train

x_train, y_train = train['comment'].values.tolist(), train['label'].values.tolist()
x_test, y_test = test['comment'].values.tolist(), test['label'].values.tolist()

print(train.shape)
print(test.shape)

(4924, 2)
(548, 2)


## TensorFlow

In [ ]:
# general config
MAX_LEN = 100
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32

EPOCHS = 3
EEVERY_EPOCH = 1000
LEARNING_RATE = 2e-5
CLIP = 0.0

MODEL_NAME_OR_PATH = 'mhshahriari/iran-bert'
OUTPUT_PATH = 'tensorflow_model.bin'

# os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

In [9]:
labels = list(sorted(train['label'].unique()))
train['label_id'] = train['label'].apply(lambda t: labels.index(t))
test['label_id'] = test['label'].apply(lambda t: labels.index(t))
label2id = {label: i for i, label in enumerate(labels)}
id2label = {v: k for k, v in label2id.items()}

In [10]:
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(MODEL_NAME_OR_PATH, **{ 'label2id': label2id, 'id2label': id2label})

tokenizer_config.json:   0%|          | 0.00/274 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/447k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

### Input Embeddings / Dataset

In [11]:
class InputExample:
    """ A single example for simple sequence classification. """
    def __init__(self, guid, text_a, text_b=None, label=None):
        """ Constructs a InputExample. """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label
def make_examples(tokenizer, x, y=None, maxlen=MAX_LEN, output_mode="classification", is_tf_dataset=True):
    examples = []
    y = y if isinstance(y, list) or isinstance(y, np.ndarray) else [None] * len(x)
    for i, (_x, _y) in tqdm(enumerate(zip(x, y)), position=0, total=len(x)):
        guid = "%s" % i
        label = int(_y)
        if isinstance(_x, str):
            text_a = _x
            text_b = None
        else:
            assert len(_x) == 2
            text_a = _x[0]
            text_b = _x[1]
        examples.append(InputExample(guid=guid, text_a=text_a, text_b=text_b, label=label))
    features = glue_convert_examples_to_features(
        examples,
        tokenizer,
        maxlen,
        output_mode=output_mode,
        label_list=list(np.unique(y)))
    all_input_ids = []
    all_attention_masks = []
    all_token_type_ids = []
    all_labels = []
    for f in tqdm(features, position=0, total=len(examples)):
        if is_tf_dataset:
            all_input_ids.append(tf.constant(f.input_ids))
            all_attention_masks.append(tf.constant(f.attention_mask))
            all_token_type_ids.append(tf.constant(f.token_type_ids))
            all_labels.append(tf.constant(f.label))
        else:
            all_input_ids.append(f.input_ids)
            all_attention_masks.append(f.attention_mask)
            all_token_type_ids.append(f.token_type_ids)
            all_labels.append(f.label)
    if is_tf_dataset:
        dataset = tf.data.Dataset.from_tensor_slices(({
            'input_ids': all_input_ids,
            'attention_mask': all_attention_masks,
            'token_type_ids': all_token_type_ids
        }, all_labels))

        return dataset, features
    xdata = [np.array(all_input_ids), np.array(all_attention_masks), np.array(all_token_type_ids)]
    ydata = all_labels
    return [xdata, ydata], features

In [12]:
train_dataset_base, train_examples = make_examples(tokenizer, x_train, np.asarray(train['label_id']), maxlen=MAX_LEN)

test_dataset_base, test_examples = make_examples(tokenizer, x_test, np.asarray(test['label_id']), maxlen=MAX_LEN)
[xtest, ytest], test_examples = make_examples(tokenizer, x_test, np.asarray(test['label_id']), maxlen=MAX_LEN, is_tf_dataset=False)

  0%|          | 0/4924 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/data/processors/glue.py:66: FutureWarning: This function will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification/run_glue.py
  warnings.warn(DEPRECATION_WARNING.format("function"), FutureWarning)


  0%|          | 0/4924 [00:00<?, ?it/s]

I0000 00:00:1745000946.472401      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


  0%|          | 0/548 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

  0%|          | 0/548 [00:00<?, ?it/s]

In [13]:
def get_training_dataset(dataset, batch_size):
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(batch_size)
    return dataset

def get_validation_dataset(dataset, batch_size):
    dataset = dataset.batch(batch_size)
    return dataset

In [14]:
train_dataset = get_training_dataset(train_dataset_base, TRAIN_BATCH_SIZE)
test_dataset = get_validation_dataset(test_dataset_base, VALID_BATCH_SIZE)

train_steps = len(train_examples) // TRAIN_BATCH_SIZE
test_steps = len(test_examples) // VALID_BATCH_SIZE

### Model

In [15]:
import math
def ceiltoup(x):
  return math.ceil(x * 100) / 100.0

f1_macro = []
accuracy = []
for i in range(10):
  # Define input layers using tf.keras.Input
  input_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_ids")
  attention_masks = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="attention_mask")
  token_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="token_type_ids")

  # Load the pre-trained BERT model
  model = TFBertModel.from_pretrained(MODEL_NAME_OR_PATH, config=config, from_pt=True)
  model.trainable = True

  # Get the BERT output
  output = model(input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids)
  
  # Apply a dense layer for classification
  output = tf.keras.layers.Dense(len(labels), activation='softmax', name='dense_out')(output['pooler_output'])  # Use tf.keras.layers.Dense

  # Create the Keras model
  model = tf.keras.Model(inputs=[input_ids, attention_masks, token_type_ids], outputs=output)  # Use tf.keras.Model

  # Compile the model
  optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
  loss = tf.keras.losses.SparseCategoricalCrossentropy()
  metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
  model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

  # Define early stopping callback
  EarlyStopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=1e-3, patience=2, verbose=0, mode='auto', restore_best_weights=True)

  # Train the model
  r = model.fit(train_dataset, validation_data=test_dataset, steps_per_epoch=train_steps, validation_steps=test_steps, epochs=20, batch_size=32, verbose=0, callbacks=[EarlyStopping_callback])

  # Make predictions
  predictions = model.predict(xtest)
  ypred = predictions.argmax(axis=-1).tolist()

  # Calculate metrics
  f1_macro.append(skm.f1_score(ytest, ypred, average="macro"))
  accuracy.append(skm.accuracy_score(ytest, ypred, normalize=True))

# Print results
print('f1 macro : ' + str(f1_macro) + '\nmean f1 macro : ' + str(ceiltoup(np.mean(f1_macro))))
print('accuracy : ' + str(accuracy) + '\nmean accuracy : ' + str(ceiltoup(np.mean(accuracy))))

pytorch_model.bin:   0%|          | 0.00/466M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertModel were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

18/18 [==============================] - 7s 129ms/step


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertModel were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

18/18 [==============================] - 8s 130ms/step


...

18/18 [==============================] - 6s 129ms/step


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertModel were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

18/18 [==============================] - 6s 129ms/step


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertModel were not initialized from the PyTorch model and are newly initialized: ['bert.pooler.dense.weight', 'bert.p

18/18 [==============================] - 6s 128ms/step
f1 macro : [0.7030890816991469, 0.7057093148060712, 0.7211542119176408, 0.6930243615617485, 0.7107138006784983, 0.7003742664736381, 0.6879697226468888, 0.7051919894296618, 0.7104664831193235, 0.7113755016710546]
mean f1 macro : 0.71
accuracy : [0.7317518248175182, 0.7354014598540146, 0.7463503649635036, 0.7281021897810219, 0.7390510948905109, 0.7281021897810219, 0.7153284671532847, 0.7372262773722628, 0.7299270072992701, 0.7354014598540146]
mean accuracy : 0.74
